목적 : unlisted data Modeling from standard scaled  
===================

import modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

In [2]:
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\Users\LG\2ndProject\비상장\3_모델링


read csv files

In [3]:
train = pd.read_csv('../datasets/Train_Test/unlisted_smoted_train_data.csv')
test = pd.read_csv('../datasets/Train_Test/unlisted_test_data.csv')

In [4]:
# train_life_cycle = train[['도입기', '성장기', '성숙기', '쇠퇴기']]
# test_life_cycle = test[['도입기', '성장기', '성숙기', '쇠퇴기']]

selected features

In [5]:
selected_features = ['CASH FLOW 대 부채비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '차입금의존도', '순운전자본비율',
                     '자기자본구성비율', '경영자본순이익률', '총자본사업이익률', '총자본영업이익률', '금융비용부담률',
                     '매출액증가율', '이윤분배율', '총자본회전률', '영업년수',
                     '도입기', '성장기', '성숙기', '쇠퇴기']
# 
# ['CASH FLOW 대 부채비율', '순운전자본비율', '자기자본구성비율', '경영자본순이익률', '총자본영업이익률', '금융비용부담률', '이윤분배율', '도입기', '성장기', '성숙기', '쇠퇴기']

x_train = train[selected_features]
x_test = test[selected_features]

y_train = train['부실판단']
y_test = test['부실판단']

test 결측치 처리

In [6]:
x_test.isna().sum().sort_values(ascending=False)
# x_test = x_test.fillna(test['영업년수'].median())

CASH FLOW 대 부채비율     0
CASH FLOW 대 총자본비율    0
성숙기                  0
성장기                  0
도입기                  0
영업년수                 0
총자본회전률               0
이윤분배율                0
매출액증가율               0
금융비용부담률              0
총자본영업이익률             0
총자본사업이익률             0
경영자본순이익률             0
자기자본구성비율             0
순운전자본비율              0
차입금의존도               0
CASH FLOW 대 매출액비율    0
쇠퇴기                  0
dtype: int64

# describe

In [7]:
x_train[train['부실판단'] == 0].describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,차입금의존도,순운전자본비율,자기자본구성비율,경영자본순이익률,총자본사업이익률,총자본영업이익률,금융비용부담률,매출액증가율,이윤분배율,총자본회전률,영업년수,도입기,성장기,성숙기,쇠퇴기
count,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000
mean,23.332367,6.287608,7.679711,35.410184,9.175669,44.210645,4.650256,5.580149,5.178156,3.545472,9.999443,5.253601,1.140162,21.107080,0.190648,0.209333,0.425021,0.174998
std,75.226438,16.324216,61.367463,24.755066,33.301547,24.181266,11.217953,9.416006,9.381163,11.976888,54.003153,111.710704,1.017161,10.111937,0.392831,0.406852,0.494370,0.379983
min,-165.260800,-42.490200,-572.428800,0.000000,-76.020400,0.720000,-49.339200,-32.002600,-32.627200,-47.190000,-97.543000,-1437.891400,0.010000,4.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.431860,-0.235627,-0.280000,13.479163,-12.345000,25.810000,0.460000,1.545692,1.170000,0.010000,-10.430000,3.955000,0.480000,13.000000,0.000000,0.000000,0.000000,0.000000
50%,6.780000,3.900000,4.570000,35.640000,8.270000,41.010000,3.090000,4.230000,3.870000,0.870000,5.130000,21.030000,0.900000,20.000000,0.000000,0.000000,0.000000,0.000000
75%,21.980000,9.415000,12.665000,54.212649,31.275000,62.185000,8.355000,8.990000,8.558613,2.640000,21.305000,40.920000,1.470000,27.000000,0.000000,0.000000,1.000000,0.000000
max,788.472000,145.442000,743.573800,92.597500,90.996800,97.423800,74.888400,50.366000,49.890000,142.110800,432.832000,86.144400,7.252600,55.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
x_train[train['부실판단'] == 1].describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,차입금의존도,순운전자본비율,자기자본구성비율,경영자본순이익률,총자본사업이익률,총자본영업이익률,금융비용부담률,매출액증가율,이윤분배율,총자본회전률,영업년수,도입기,성장기,성숙기,쇠퇴기
count,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000,10543.000000
mean,-1.668247,-0.222658,-23.029462,51.925177,-13.634209,27.392592,-4.017969,-2.275643,-2.679081,12.430178,40.517746,-83.269907,0.726382,22.725579,0.190648,0.209333,0.425021,0.174998
std,27.735804,10.488194,182.345917,26.320933,35.370808,23.204598,12.994293,8.044580,7.986313,33.486497,197.637264,333.510357,0.742422,11.253837,0.392831,0.406852,0.494370,0.379983
min,-348.859200,-66.805600,-2174.418800,0.000000,-121.618400,0.000000,-96.701400,-60.590300,-60.681000,-138.847200,-91.373200,-4063.945200,0.010000,4.000000,0.000000,0.000000,0.000000,0.000000
25%,-4.570000,-3.150000,-7.215000,33.890366,-35.600222,8.293543,-6.440000,-3.825851,-4.090000,1.080000,-10.930000,-50.942432,0.180000,14.000000,0.000000,0.000000,0.000000,0.000000
50%,1.100000,0.799901,1.735632,53.460000,-12.740000,22.860000,-1.736403,-0.420000,-0.780000,3.227992,1.860863,-4.537348,0.544352,21.000000,0.000000,0.000000,0.000000,0.000000
75%,5.709451,4.133856,8.772211,70.180000,7.063614,40.738297,0.480000,1.403932,0.992218,10.973135,20.154591,4.197710,0.980000,28.639687,0.000000,0.000000,1.000000,0.000000
max,119.656000,54.232400,315.224300,124.008000,89.081700,97.928000,75.338400,21.816000,21.375600,350.081900,2135.799400,91.481600,5.049800,62.970000,1.000000,1.000000,1.000000,1.000000


# standard scaling

In [9]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

labels = x_train.columns

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(data=x_train, columns = labels)
x_test = pd.DataFrame(data=x_test, columns = labels)

# modeling

In [13]:
validation = pd.DataFrame()
default = pd.DataFrame()
best = pd.DataFrame()

## LogisticRegression

기본 모델

In [14]:
# 모델 생성
logit_model = LogisticRegression()

# 모델 학습 및 평가
logit_model.fit(x_train, y_train)
y_pred = logit_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 스코어: {f1:.3f}')
print(f'ROC AUC 스코어: {roc_auc:.3f}')

# 결과 저장
result = {
    'Accuracy' : round(accuracy, 3),
    'Precision' : round(precision, 3),
    'Recall' : round(recall, 3),
    'F1' : round(f1, 3),
    'ROC AUC' : round(roc_auc, 3)
}
default['LogisticRegression'] = result

Accuracy: 0.746
Precision: 0.641
Recall: 0.704
F1 스코어: 0.671
ROC AUC 스코어: 0.737


파라미터 조정

In [15]:
# 하이퍼파라미터 범위 지정
param_grid = {
    'C': np.arange(0, 100.0, 0.01),  # 규제 강도
    'penalty': ['l1', 'l2', 'elasticnet'],  # 규제 유형
    'solver': ['liblinear', 'saga']  # 최적화 알고리즘
}

model = LogisticRegression()
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=99,
                                   scoring='accuracy', cv=7, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_

y_pred = best_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Logistic 하이퍼파라미터 조정 평가 지표")
print("Best Parameters:", random_search.best_params_)
print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'ROC AUC Score: {roc_auc:.3f}')

# 파라미터 조절 결과
result0_logit = {
    'Accuracy' : round(accuracy, 3),
    'Precision' : round(precision, 3),
    'Recall' : round(recall, 3),
    'ROC AUC' : round(roc_auc, 3),
    'Best Parameters' : random_search.best_params_
}
best['LogisticRegression'] = result0_logit


Fitting 7 folds for each of 99 candidates, totalling 693 fits
Logistic 하이퍼파라미터 조정 평가 지표
Best Parameters: {'solver': 'saga', 'penalty': 'l1', 'C': 2.11}
Accuracy: 0.747
Precision: 0.642
Recall: 0.704
F1 Score: 0.672
ROC AUC Score: 0.738


최적 파라미터 검증

In [16]:
# 모델 생성
cv_logit_model = LogisticRegression(
    # solver=random_search.best_params_['solver'], 
    # penalty=random_search.best_params_['penalty'],
    # C=random_search.best_params_['C']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='accuracy')
cv_precision = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='precision')
cv_recall = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='recall')
cv_f1 = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='f1')
cv_roc_auc = cross_val_score(cv_logit_model, x_train, y_train, cv=7, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_logit = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['LogisticRegression'] = cv_result_logit

CV_Accuracy_Scores: [0.69532028 0.77962164 0.82702523 0.82669323 0.83333333 0.81042497
 0.67031873]
CV_Precision_Scores: [0.71151079 0.78383838 0.80955374 0.7932062  0.79952267 0.78111846
 0.68736304]
CV_Recall_Scores: [0.65670651 0.77239549 0.85524568 0.88379814 0.88977424 0.8625498
 0.624834  ]
CV_F1_Scores: [0.68301105 0.77807487 0.83177268 0.83605528 0.84223759 0.81981698
 0.6546087 ]
CV_ROC/AUC: [0.75503824 0.85535099 0.90957551 0.9048926  0.91049172 0.88108302
 0.74114653]

=======교차검증 결과=======
CV_Accuracy_mean: 0.778
CV_Precision_mean: 0.767
CV_Recall_mean: 0.792
CV_F1_스코어_mean: 0.778
CV_ROC_AUC+스코어_mean: 0.851


## RandomForestClassifier

기본 모델

In [18]:
# Random Forest 모델 생성 및 학습
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_rf = rf_model.predict(x_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)

print("Random Forest 모델 평가 지표")
print(f'Accuracy: {accuracy_rf:.3f}')
print(f'Precision: {precision_rf:.3f}')
print(f'Recall: {recall_rf:.3f}')
print(f'F1 스코어: {f1_rf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rf:.3f}')

# 기본결과 저장
result_rf = {
    'Accuracy' : round(accuracy_rf, 3),
    'Precision' : round(precision_rf, 3),
    'Recall' : round(recall_rf, 3),
    'F1' : round(f1_rf, 3),
    'ROC AUC' : round(roc_auc_rf, 3)
}
default['RandomForestClassifier'] = result_rf
default


Random Forest 모델 평가 지표
Accuracy: 0.791
Precision: 0.690
Recall: 0.785
F1 스코어: 0.734
ROC AUC 스코어: 0.790


,LogisticRegression,RandomForestClassifier
Accuracy,0.693,0.791
Precision,0.572,0.690
Recall,0.658,0.785
F1,0.612,0.734
ROC AUC,0.686,0.790


파라미터 조정

In [19]:
random_search = {'n_estimators': np.arange(10, 200, 10),
                 'max_depth': np.arange(1, 10, 1),
                 'min_samples_split': np.arange(1, 9, 1),
                 'min_samples_leaf': np.arange(1, 9, 1)}

clf = RandomForestClassifier()
random_rf = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 10,
                               cv=5, verbose= 1, random_state= 101, n_jobs = -1)
random_rf.fit(x_train,y_train)
random_pf = random_rf.best_estimator_.predict(x_test)

accuracy_rdrf = accuracy_score(y_test, random_pf)
precision_rdrf = precision_score(y_test, random_pf)
recall_rdrf = recall_score(y_test, random_pf)
f1_rdrf = f1_score(y_test, random_pf)
roc_auc_rdrf = roc_auc_score(y_test, random_pf)

print("Random Forest 모델 하이퍼파라미터 조정 평가 지표")
print(random_rf.best_params_)
print(f'Accuracy: {accuracy_rdrf:.3f}')
print(f'Precision: {precision_rdrf:.3f}')
print(f'Recall: {recall_rdrf:.3f}')
print(f'F1 스코어: {f1_rdrf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rdrf:.3f}')

# 파라미터 조정 결과 저장
result0_rf = {
    'Accuracy' : round(accuracy_rdrf, 3),
    'Precision' : round(precision_rdrf, 3),
    'Recall' : round(recall_rdrf, 3),
    'F1' : round(f1_rdrf, 3),
    'ROC AUC' : round(roc_auc_rdrf, 3),
    'Best Parameters' : random_rf.best_params_
}
best['RandomForestClassifier'] = result0_rf
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Random Forest 모델 하이퍼파라미터 조정 평가 지표
{'n_estimators': 50, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_depth': 6}
Accuracy: 0.792
Precision: 0.673
Recall: 0.845
F1 스코어: 0.749
ROC AUC 스코어: 0.803


,LogisticRegression,RandomForestClassifier
Accuracy,0.693,0.792
Precision,0.572,0.673
Recall,0.658,0.845
ROC AUC,0.686,0.803
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 50, 'min_samples_split': 4, '..."


최적 파라미터 검증

In [20]:
random_rf.best_params_['n_estimators']

50

In [21]:
# 모델 생성
cv_model_rf = RandomForestClassifier(
    n_estimators=random_rf.best_params_['n_estimators'],
    min_samples_split=random_rf.best_params_['min_samples_split'],
    min_samples_leaf=random_rf.best_params_['min_samples_leaf'],
    max_depth=random_rf.best_params_['max_depth']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_rf, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_rf = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['RandomForestClassifier'] = cv_result_rf

CV_Accuracy_Scores: [0.68622894 0.79140035 0.88372093 0.7627907  0.66511628]
CV_Precision_Scores: [0.64792627 0.74924471 0.86710963 0.73333333 0.63291139]
CV_Recall_Scores: [0.82325581 0.86643438 0.89651163 0.83139535 0.8244186 ]
CV_F1_Scores: [0.72108844 0.80217391 0.88388215 0.77971647 0.71223378]
CV_ROC/AUC: [0.78260878 0.8758515  0.94690779 0.86408599 0.74888521]

=======교차검증 결과=======
CV_Accuracy_mean: 0.758
CV_Precision_mean: 0.726
CV_Recall_mean: 0.848
CV_F1_스코어_mean: 0.780
CV_ROC_AUC+스코어_mean: 0.844


## AdaBoostClassifier

기본 모델

In [22]:
# AdaBoost 모델 생성 및 학습
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_adaboost = adaboost_model.predict(x_test)

accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)

print("AdaBoost 모델 평가 지표:")
print(f'Accuracy: {accuracy_adaboost:.3f}')
print(f'Precision: {precision_adaboost:.3f}')
print(f'Recall: {recall_adaboost:.3f}')
print(f'F1 스코어: {f1_adaboost:.3f}')
print(f'ROC AUC 스코어: {roc_auc_adaboost:.3f}')

AdaBoost 모델 평가 지표:
Accuracy: 0.767
Precision: 0.666
Recall: 0.736
F1 스코어: 0.699
ROC AUC 스코어: 0.761


In [23]:
# 기본 결과
result = {
    'Accuracy' : round(accuracy_adaboost, 3),
    'Precision' : round(precision_adaboost, 3),
    'Recall' : round(recall_adaboost, 3),
    'F1' : round(f1_adaboost, 3),
    'ROC AUC' : round(roc_auc_adaboost, 3)
}
default['AdaBoostClassifier'] = result
default

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier
Accuracy,0.693,0.791,0.767
Precision,0.572,0.690,0.666
Recall,0.658,0.785,0.736
F1,0.612,0.734,0.699
ROC AUC,0.686,0.790,0.761


파라미터 조정

In [24]:
# randon search AdaBoostClassifier
adaboost_model = AdaBoostClassifier()

param_grid = {
    'n_estimators': np.arange(10, 300, 10),  # 트리 개수
    'learning_rate': np.arange(0.01, 1, 0.01)  # 학습률
}

random_search = RandomizedSearchCV(estimator=adaboost_model, param_distributions=param_grid, n_iter=10,
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_

y_pred_adaboost = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
precision_adaboost = precision_score(y_test, y_pred_adaboost)
recall_adaboost = recall_score(y_test, y_pred_adaboost)
f1_adaboost = f1_score(y_test, y_pred_adaboost)
roc_auc_adaboost = roc_auc_score(y_test, y_pred_adaboost)

print("Best Parameters:", random_search.best_params_)
print("AdaBoost 모델 하이퍼파라미터 조정 평가 지표:")
print(f'Accuracy: {accuracy_adaboost:.3f}')
print(f'Precision: {precision_adaboost:.3f}')
print(f'Recall: {recall_adaboost:.3f}')
print(f'F1 Score: {f1_adaboost:.3f}')
print(f'ROC AUC Score: {roc_auc_adaboost:.3f}')

# 파라미터 조정 결과 저장
result0_adaboost = {
    'Accuracy' : round(accuracy_adaboost, 3),
    'Precision' : round(precision_adaboost, 3),
    'Recall' : round(recall_adaboost, 3),
    'F1' : round(f1_adaboost, 3),
    'ROC AUC' : round(roc_auc_adaboost, 3),
    'Best Parameters' : random_search.best_params_
}
best['AdaBoostClassifier'] = result0_adaboost
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Best Parameters: {'n_estimators': 240, 'learning_rate': 0.75}
AdaBoost 모델 하이퍼파라미터 조정 평가 지표:
Accuracy: 0.772
Precision: 0.669
Recall: 0.750
F1 Score: 0.707
ROC AUC Score: 0.767


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier
Accuracy,0.693,0.792,0.772
Precision,0.572,0.673,0.669
Recall,0.658,0.845,0.75
ROC AUC,0.686,0.803,0.767
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 50, 'min_samples_split': 4, '...","{'n_estimators': 240, 'learning_rate': 0.75}"


최적 파라미터 검증

In [25]:
# 모델 생성
# AdaBoostClassifier 최적파라미터 적용 Crossvalidation
cv_model_adaboost = AdaBoostClassifier(
    n_estimators=random_search.best_params_['n_estimators'],
    learning_rate=random_search.best_params_['learning_rate']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_adaboost, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_adaboost = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['AdaBoostClassifier'] = cv_result_adaboost

CV_Accuracy_Scores: [0.70772807 0.7797792  0.84825581 0.78139535 0.70697674]
CV_Precision_Scores: [0.70124014 0.78690476 0.86391252 0.77069351 0.69517544]
CV_Recall_Scores: [0.72325581 0.76771196 0.82674419 0.80116279 0.7372093 ]
CV_F1_Scores: [0.71207785 0.77718989 0.84491979 0.78563284 0.71557562]
CV_ROC/AUC: [0.79977919 0.87399454 0.92499121 0.86802866 0.79264129]

=======교차검증 결과=======
CV_Accuracy_mean: 0.765
CV_Precision_mean: 0.764
CV_Recall_mean: 0.771
CV_F1_스코어_mean: 0.767
CV_ROC_AUC+스코어_mean: 0.852


## BaggingClassifier

기본 모델

In [26]:
# Bagging 모델 생성 및 학습
bagging_model = BaggingClassifier()
bagging_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_bagging = bagging_model.predict(x_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging)
recall_bagging = recall_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
roc_auc_bagging = roc_auc_score(y_test, y_pred_bagging)

print("Bagging 모델 평가 지표:")
print(f'Accuracy: {accuracy_bagging:.3f}')
print(f'Precision: {precision_bagging:.3f}')
print(f'Recall: {recall_bagging:.3f}')
print(f'F1 스코어: {f1_bagging:.3f}')
print(f'ROC AUC 스코어: {roc_auc_bagging:.3f}')

# 결과 저장
# 기본값 저장
result_bagging = {
    'Accuracy' : round(accuracy_bagging, 3),
    'Precision' : round(precision_bagging, 3),
    'Recall' : round(recall_bagging, 3),
    'F1' : round(f1_bagging, 3),
    'ROC AUC' : round(roc_auc_bagging, 3)
}
default['BaggingClassifier'] = result_bagging
default

Bagging 모델 평가 지표:
Accuracy: 0.778
Precision: 0.698
Recall: 0.698
F1 스코어: 0.698
ROC AUC 스코어: 0.761


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
Accuracy,0.693,0.791,0.767,0.778
Precision,0.572,0.690,0.666,0.698
Recall,0.658,0.785,0.736,0.698
F1,0.612,0.734,0.699,0.698
ROC AUC,0.686,0.790,0.761,0.761


파라미터 조정

In [27]:
# random search
bagging_model = BaggingClassifier()

param_grid = {
    'n_estimators': np.arange(10, 100, 10),
    'max_samples': np.arange(0.1, 1.0, 0.1),
    'max_features': np.arange(0.1, 1, 0.1)
}

random_search = RandomizedSearchCV(estimator=bagging_model, param_distributions=param_grid, n_iter=10,
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

# 최적의 모델 저장
best_model = random_search.best_estimator_

# 최적의 모델로 예측 수행
y_pred_bagging = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging)
recall_bagging = recall_score(y_test, y_pred_bagging)
f1_bagging = f1_score(y_test, y_pred_bagging)
roc_auc_bagging = roc_auc_score(y_test, y_pred_bagging)

print("Best Parameters:", random_search.best_params_)
print("Bagging 모델 하이퍼파라미터 평가 지표:")
print(f'Accuracy: {accuracy_bagging:.3f}')
print(f'Precision: {precision_bagging:.3f}')
print(f'Recall: {recall_bagging:.3f}')
print(f'F1 Score: {f1_bagging:.3f}')
print(f'ROC AUC Score: {roc_auc_bagging:.3f}')

# 파라미터 조정 결과 저장
result0_bagging = {
    'Accuracy' : round(accuracy_bagging, 3),
    'Precision' : round(precision_bagging, 3),
    'Recall' : round(recall_bagging, 3),
    'F1' : round(f1_bagging, 3),
    'ROC AUC' : round(roc_auc_bagging, 3),
    'Best Parameters' : random_search.best_params_
}
best['BaggingClassifier'] = result0_bagging
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Best Parameters: {'n_estimators': 30, 'max_samples': 0.4, 'max_features': 0.7000000000000001}
Bagging 모델 하이퍼파라미터 평가 지표:
Accuracy: 0.782
Precision: 0.687
Recall: 0.751
F1 Score: 0.717
ROC AUC Score: 0.776


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier
Accuracy,0.693,0.792,0.772,0.782
Precision,0.572,0.673,0.669,0.687
Recall,0.658,0.845,0.75,0.751
ROC AUC,0.686,0.803,0.767,0.776
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 50, 'min_samples_split': 4, '...","{'n_estimators': 240, 'learning_rate': 0.75}","{'n_estimators': 30, 'max_samples': 0.4, 'max_..."


최적 파라미터 검증

In [28]:
# 모델 생성
# BaggingClassifier 최적파라미터 적용 Crossvalidation
cv_model_bagging = BaggingClassifier(
    n_estimators=random_search.best_params_['n_estimators'],
    max_samples=random_search.best_params_['max_samples'],
    max_features=random_search.best_params_['max_features']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_bagging, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_bagging = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['BaggingClassifier'] = cv_result_bagging

CV_Accuracy_Scores: [0.68564788 0.77861708 0.87906977 0.75697674 0.67267442]
CV_Precision_Scores: [0.67114788 0.78099652 0.88516746 0.76218097 0.65079365]
CV_Recall_Scores: [0.76162791 0.78745645 0.91046512 0.78604651 0.70581395]
CV_F1_Scores: [0.68482039 0.78265896 0.86935867 0.76949345 0.67157418]
CV_ROC/AUC: [0.77310523 0.86861208 0.94358707 0.85331057 0.74156301]

=======교차검증 결과=======
CV_Accuracy_mean: 0.755
CV_Precision_mean: 0.750
CV_Recall_mean: 0.790
CV_F1_스코어_mean: 0.756
CV_ROC_AUC+스코어_mean: 0.836


## SVC_linear

SVC linear 기본모델

In [29]:
# SVM 모델 생성 및 학습
svm_model = SVC(kernel='linear')
svm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_svm = svm_model.predict(x_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 스코어: {f1_svm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_svm:.3f}')

# 결과 저장
result_linear = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3)
}
{}
default['SVC_linear'] = result_linear
default

SVM 모델 평가 지표:
Accuracy: 0.692
Precision: 0.571
Recall: 0.658
F1 스코어: 0.611
ROC AUC 스코어: 0.685


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear
Accuracy,0.693,0.791,0.767,0.778,0.692
Precision,0.572,0.690,0.666,0.698,0.571
Recall,0.658,0.785,0.736,0.698,0.658
F1,0.612,0.734,0.699,0.698,0.611
ROC AUC,0.686,0.790,0.761,0.761,0.685


SVC linear 파라미터 조정

In [10]:
svm_model = SVC()

param_grid = {
    'C': np.arange(0.1, 10, 0.1),  # 규제 매개변수
    'gamma': [0.1, 0.01, 0.001, 0.0001],  # 커널 계수
    'kernel': ['linear']  # 커널 타입
}

random_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=10,
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

print("Best Parameters:", random_search.best_params_)
best_model = random_search.best_estimator_

y_pred_svm = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

# 평가 메트릭 출력
print("SVM 모델 평가 지표:")
print("Best Parameters:", random_search.best_params_)
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 Score: {f1_svm:.3f}')
print(f'ROC AUC Score: {roc_auc_svm:.3f}')

# 파라미터 조정 결과 저장
result0_svc_linear = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3),
    'Best Parameters' : random_search.best_params_
}
best['SVC_linear'] = result0_svc_linear
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Best Parameters: {'kernel': 'linear', 'gamma': 0.1, 'C': 3.9000000000000004}
SVM 모델 평가 지표:
Best Parameters: {'kernel': 'linear', 'gamma': 0.1, 'C': 3.9000000000000004}
Accuracy: 0.747
Precision: 0.639
Recall: 0.715
F1 Score: 0.675
ROC AUC Score: 0.740


NameError: name 'best' is not defined

SVC linear 최적 파라미터 검증

In [31]:
# 모델 생성
# SVC_linear 최적파라미터 적용 Crossvalidation
cv_model_scv_linear = SVC(
    kernel='linear',
    gamma=random_search.best_params_['gamma'],
    C=random_search.best_params_['C']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_scv_linear, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_linear = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['SVC_linear'] = cv_result_linear
validation

CV_Accuracy_Scores: [0.67112144 0.73038931 0.77325581 0.71337209 0.63023256]
CV_Precision_Scores: [0.69811321 0.73772455 0.73833671 0.70971429 0.64395887]
CV_Recall_Scores: [0.60232558 0.71544715 0.84651163 0.72209302 0.58255814]
CV_F1_Scores: [0.64669164 0.72641509 0.78873239 0.71585014 0.61172161]
CV_ROC/AUC: [0.71930962 0.80740351 0.86265955 0.78176582 0.70035154]

=======교차검증 결과=======
CV_Accuracy_mean: 0.704
CV_Precision_mean: 0.706
CV_Recall_mean: 0.694
CV_F1_스코어_mean: 0.698
CV_ROC_AUC+스코어_mean: 0.774


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear
Accuracy,0.705,0.758,0.765,0.755,0.704
Precision,0.705,0.726,0.764,0.750,0.706
Recall,0.700,0.848,0.771,0.790,0.694
F1,0.701,0.780,0.767,0.756,0.698
ROC AUC,0.775,0.844,0.852,0.836,0.774


## SVC_RBF

In [32]:
# SVM 모델 생성 및 학습
svm_model = SVC(kernel='rbf')
svm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_svm = svm_model.predict(x_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

print("SVM 모델 평가 지표:")
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 스코어: {f1_svm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_svm:.3f}')

result_svc_rbf = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3)
}
default['SVC_rbf'] = result_svc_rbf
default

SVM 모델 평가 지표:
Accuracy: 0.759
Precision: 0.650
Recall: 0.747
F1 스코어: 0.695
ROC AUC 스코어: 0.756


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf
Accuracy,0.693,0.791,0.767,0.778,0.692,0.759
Precision,0.572,0.690,0.666,0.698,0.571,0.650
Recall,0.658,0.785,0.736,0.698,0.658,0.747
F1,0.612,0.734,0.699,0.698,0.611,0.695
ROC AUC,0.686,0.790,0.761,0.761,0.685,0.756


파라미터 조정

In [33]:
svm_model = SVC()

param_grid = {
    'C': np.arange(0.0001, 10, 0.0001),  # Adjust the maximum value
    'gamma': np.arange(0.0001, 1, 0.0001),  # Keep the range as is
    'kernel': ['rbf']
}

random_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_grid, n_iter=10,
                                   scoring='accuracy', cv=5, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

print("Best Parameters:", random_search.best_params_)
best_model = random_search.best_estimator_

y_pred_svm = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)

# 평가 메트릭 출력
print("SVM 모델 평가 지표:")
print("Best Parameters:", random_search.best_params_)
print(f'Accuracy: {accuracy_svm:.3f}')
print(f'Precision: {precision_svm:.3f}')
print(f'Recall: {recall_svm:.3f}')
print(f'F1 Score: {f1_svm:.3f}')
print(f'ROC AUC Score: {roc_auc_svm:.3f}')

# 파라미터 조정 결과 저장
result0_svc_rbf = {
    'Accuracy' : round(accuracy_svm, 3),
    'Precision' : round(precision_svm, 3),
    'Recall' : round(recall_svm, 3),
    'F1' : round(f1_svm, 3),
    'ROC AUC' : round(roc_auc_svm, 3),
    'Best Parameters' : random_search.best_params_
}
best['SVC_rbf'] = result0_svc_rbf
best

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters: {'kernel': 'rbf', 'gamma': 0.058800000000000005, 'C': 1.9993}
SVM 모델 평가 지표:
Best Parameters: {'kernel': 'rbf', 'gamma': 0.058800000000000005, 'C': 1.9993}
Accuracy: 0.760
Precision: 0.653
Recall: 0.744
F1 Score: 0.695
ROC AUC Score: 0.757


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf
Accuracy,0.693,0.792,0.772,0.782,0.692,0.76
Precision,0.572,0.673,0.669,0.687,0.571,0.653
Recall,0.658,0.845,0.75,0.751,0.658,0.744
ROC AUC,0.686,0.803,0.767,0.776,0.685,0.757
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 50, 'min_samples_split': 4, '...","{'n_estimators': 240, 'learning_rate': 0.75}","{'n_estimators': 30, 'max_samples': 0.4, 'max_...","{'kernel': 'linear', 'gamma': 0.1, 'C': 0.1}","{'kernel': 'rbf', 'gamma': 0.05880000000000000..."


svc rbf 최적 파라미터 검증

In [34]:
# 모델 생성
# SVC_linear 최적파라미터 적용 Crossvalidation
cv_model_scv_rbf = SVC(
    kernel='rbf',
    gamma=random_search.best_params_['gamma'],
    C=random_search.best_params_['C']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_scv_rbf, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_rbf = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['SVC_rbf'] = cv_result_rbf

CV_Accuracy_Scores: [0.70482278 0.77280651 0.87848837 0.75348837 0.6627907 ]
CV_Precision_Scores: [0.70465116 0.77453271 0.84227129 0.75       0.66018307]
CV_Recall_Scores: [0.70465116 0.77003484 0.93139535 0.76046512 0.67093023]
CV_F1_Scores: [0.70465116 0.77227723 0.88459415 0.7551963  0.66551326]
CV_ROC/AUC: [0.77662669 0.86286227 0.93626014 0.84229448 0.745457  ]

=======교차검증 결과=======
CV_Accuracy_mean: 0.754
CV_Precision_mean: 0.746
CV_Recall_mean: 0.767
CV_F1_스코어_mean: 0.756
CV_ROC_AUC+스코어_mean: 0.833


## LGBMClassifier

기본 모델

In [35]:
# LGBM 모델 생성 및 학습
lgbm_model = LGBMClassifier()
lgbm_model.fit(x_train, y_train)

# 예측 및 성능 평가
y_pred_lgbm = lgbm_model.predict(x_test)

accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)
precision_lgbm = precision_score(y_test, y_pred_lgbm)
recall_lgbm = recall_score(y_test, y_pred_lgbm)
f1_lgbm = f1_score(y_test, y_pred_lgbm)
roc_auc_lgbm = roc_auc_score(y_test, y_pred_lgbm)

print("LightGBM 모델 평가 지표:")
print(f'Accuracy: {accuracy_lgbm:.3f}')
print(f'Precision: {precision_lgbm:.3f}')
print(f'Recall: {recall_lgbm:.3f}')
print(f'F1 스코어: {f1_lgbm:.3f}')
print(f'ROC AUC 스코어: {roc_auc_lgbm:.3f}')

result_lgbm = {
    'Accuracy' : round(accuracy_lgbm, 3),
    'Precision' : round(precision_lgbm, 3),
    'Recall' : round(recall_lgbm, 3),
    'F1' : round(f1_lgbm, 3),
    'ROC AUC' : round(roc_auc_lgbm, 3)
}
default['LGBMClassifier'] = result_lgbm
default

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4301, number of negative: 4301
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001419 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1797
[LightGBM] [Info] Number of data points in the train set: 8602, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LightGBM 모델 평가 지표:
Accuracy: 0.792
Precision: 0.690
Recall: 0.789
F1 스코어: 0.736
ROC AUC 스코어: 0.791


,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier
Accuracy,0.693,0.791,0.767,0.778,0.692,0.759,0.792
Precision,0.572,0.690,0.666,0.698,0.571,0.650,0.690
Recall,0.658,0.785,0.736,0.698,0.658,0.747,0.789
F1,0.612,0.734,0.699,0.698,0.611,0.695,0.736
ROC AUC,0.686,0.790,0.761,0.761,0.685,0.756,0.791


In [22]:
# 파라미터 조정
param_grid = {
        'learning_rate': [0.001, 0.01, 0.1, 1],
        'n_estimators': np.arange(1, 100, 1),
        'max_depth': np.arange(1, 100, 1),
        'num_leaves': np.arange(1, 100, 1),
        'min_child_samples': np.arange(1, 100, 1)
    }

lgbm = LGBMClassifier()
random = RandomizedSearchCV(lgbm, param_grid, cv = 5, n_jobs = -1, n_iter=999)
random.fit(x_train,y_train)
random_pf = random.best_estimator_.predict(x_test)

accuracy_rdrf = accuracy_score(y_test, random_pf)
precision_rdrf = precision_score(y_test, random_pf)
recall_rdrf = recall_score(y_test, random_pf)
f1_rdrf = f1_score(y_test, random_pf)
roc_auc_rdrf = roc_auc_score(y_test, random_pf)

print("LGBM 모델 하이퍼파라미터 조정 평가 지표")
print(random.best_params_)
print(f'Accuracy: {accuracy_rdrf:.3f}')
print(f'Precision: {precision_rdrf:.3f}')
print(f'Recall: {recall_rdrf:.3f}')
print(f'F1 스코어: {f1_rdrf:.3f}')
print(f'ROC AUC 스코어: {roc_auc_rdrf:.3f}')

# 파라미터 조정 결과 저장
result0_lgbm = {
    'Accuracy' : round(accuracy_rdrf, 3),
    'Precision' : round(precision_rdrf, 3),
    'Recall' : round(recall_rdrf, 3),
    'F1' : round(f1_rdrf, 3),
    'ROC AUC' : round(roc_auc_rdrf, 3),
    'Best Parameters' : random.best_params_
}
best['LGBMClassifier'] = result0_lgbm
best

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 10543, number of negative: 10543
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002902 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3580
[LightGBM] [Info] Number of data points in the train set: 21086, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LGBM 모델 하이퍼파라미터 조정 평가 지표
{'num_leaves': 97, 'n_estimators': 97, 'min_child_samples': 75, 'max_depth': 48, 'learning_rate': 0.1}
Accuracy: 0.818
Precision: 0.733
Recall: 0.794
F1 스코어: 0.763
ROC AUC 스코어: 0.813


,LogisticRegression,LGBMClassifier
Accuracy,0.747,0.818
Precision,0.642,0.733
Recall,0.704,0.794
ROC AUC,0.738,0.813
Best Parameters,"{'solver': 'saga', 'penalty': 'l1', 'C': 2.11}","{'num_leaves': 97, 'n_estimators': 97, 'min_ch..."


LGBM 최적 파라미터 검증

In [17]:
# 모델 생성
# lgbm 최적파라미터 적용 Crossvalidation
cv_model_lgbm = LGBMClassifier(
    # num_leaves=random.best_params_['num_leaves'],
    # n_estimators=random.best_params_['n_estimators'],
    # max_depth=random.best_params_['max_depth'],
    # learning_rate=random.best_params_['learning_rate']
)

# Cross Validation
cv_accuracy = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(cv_model_lgbm, x_train, y_train, cv=5, scoring='roc_auc')

print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 결과=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC+스코어_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_lgbm = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['LGBMClassifier'] = cv_result_lgbm

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 8434, number of negative: 8434
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003562 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3578
[LightGBM] [Info] Number of data points in the train set: 16868, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 8435, number of negative: 8434
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002317 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3579
[LightGBM] [Info] Number of data points in the train set: 16869, number of

#### 딥러닝

In [38]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# import tensorflow as tf

In [39]:
# model = Sequential()
# model.add(Dense(256, input_dim = 12, activation = 'relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation = 'relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation = 'sigmoid'))

In [40]:
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# history = model.fit(x_train, y_train, epochs=200, batch_size=5)

# print("\n Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

In [41]:
# # 모델을 사용하여 테스트 데이터에 대한 예측 수행
# y_pred = model.predict(x_test)
# y_pred = binarize(y_pred, threshold=0.5)  # 예측값을 0.5 임계값을 기준으로 이진 분류로 변환

# # 정확도(accuracy) 계산
# accuracy = accuracy_score(y_test, y_pred)
# print(f"accuracy: {accuracy:.4f}")

# # F1 점수(f1 score) 계산
# f1 = f1_score(y_test, y_pred)
# print(f"F1-Score: {f1:.4f}")

# # 재현율(recall) 계산
# recall = recall_score(y_test, y_pred)
# print(f"recall: {recall:.4f}")

# # 정밀도(precision) 계산
# precision = precision_score(y_test, y_pred)
# print(f"precision: {precision:.4f}")

## XGboost classifier

In [23]:
# 파라미터 조정
xgboost_model = XGBClassifier()

param_grid = {
    'n_estimators': np.arange(1, 100, 1),  # 트리 개수
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1]  # 학습률
}

random_search = RandomizedSearchCV(estimator=xgboost_model, param_distributions=param_grid, n_iter=999,
                                   scoring='accuracy', cv=9, verbose=1, random_state=42)

random_search.fit(x_train, y_train)

best_model = random_search.best_estimator_

y_pred_xgboost = best_model.predict(x_test)

# 평가 메트릭 계산
accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
precision_xgboost = precision_score(y_test, y_pred_xgboost)
recall_xgboost = recall_score(y_test, y_pred_xgboost)
f1_xgboost = f1_score(y_test, y_pred_xgboost)
roc_auc_xgboost = roc_auc_score(y_test, y_pred_xgboost)

print("Best Parameters:", random_search.best_params_)
print("XGBoost 모델 하이퍼파라미터 조정 평가 지표:")
print(f'Accuracy: {accuracy_xgboost:.3f}')
print(f'Precision: {precision_xgboost:.3f}')
print(f'Recall: {recall_xgboost:.3f}')
print(f'F1 Score: {f1_xgboost:.3f}')
print(f'ROC AUC Score: {roc_auc_xgboost:.3f}')

result0_xgboost = {
    'Accuracy' : round(accuracy_xgboost, 3),
    'Precision' : round(precision_xgboost, 3),
    'Recall' : round(recall_xgboost, 3),
    'F1' : round(f1_xgboost, 3),
    'ROC AUC' : round(roc_auc_xgboost, 3),
    'Best Parameters' : random_search.best_params_
}
best['LGBMClassifier'] = result0_xgboost
best

Fitting 9 folds for each of 495 candidates, totalling 4455 fits


Best Parameters: {'n_estimators': 99, 'learning_rate': 0.1}
XGBoost 모델 하이퍼파라미터 조정 평가 지표:
Accuracy: 0.813
Precision: 0.720
Recall: 0.808
F1 Score: 0.761
ROC AUC Score: 0.812


,LogisticRegression,LGBMClassifier
Accuracy,0.747,0.813
Precision,0.642,0.72
Recall,0.704,0.808
ROC AUC,0.738,0.812
Best Parameters,"{'solver': 'saga', 'penalty': 'l1', 'C': 2.11}","{'n_estimators': 99, 'learning_rate': 0.1}"


In [43]:
# 최적 파라미터 검증
xgboost_model = XGBClassifier(
    n_estimators=random_search.best_params_['n_estimators'],
    learning_rate=random_search.best_params_['learning_rate']
)

# Cross Validation
cv_accuracy = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='accuracy')
cv_precision = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='precision')
cv_recall = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='recall')
cv_f1 = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='f1')
cv_roc_auc = cross_val_score(xgboost_model, x_train, y_train, cv=5, scoring='roc_auc')

print('=======교차검증 결과=======')
print("CV_Accuracy_Scores:", cv_accuracy)
print("CV_Precision_Scores:", cv_precision)
print("CV_Recall_Scores:", cv_recall)
print("CV_F1_Scores:", cv_f1)
print("CV_ROC/AUC:", cv_roc_auc)

print('\n=======교차검증 평균값=======')
print(f'CV_Accuracy_mean: {cv_accuracy.mean():.3f}')
print(f'CV_Precision_mean: {cv_precision.mean():.3f}')
print(f'CV_Recall_mean: {cv_recall.mean():.3f}')
print(f'CV_F1_스코어_mean: {cv_f1.mean():.3f}')
print(f'CV_ROC_AUC_mean: {cv_roc_auc.mean():.3f}')

# cv 결과 저장
cv_result_xgboost = {
    'Accuracy' : round(cv_accuracy.mean(), 3),
    'Precision': round(cv_precision.mean(), 3),
    'Recall' : round(cv_recall.mean(), 3),
    'F1' : round(cv_f1.mean(), 3),
    'ROC AUC' : round(cv_roc_auc.mean(), 3)
}
validation['XGBClassifier'] = cv_result_xgboost

=======교차검증 결과=======
CV_Accuracy_Scores: [0.72399768 0.80766996 0.88837209 0.78837209 0.69651163]
CV_Precision_Scores: [0.7054429  0.78433476 0.86225597 0.77192982 0.67315574]
CV_Recall_Scores: [0.76860465 0.84901278 0.9244186  0.81860465 0.76395349]
CV_F1_Scores: [0.73567056 0.8153932  0.89225589 0.79458239 0.71568627]
CV_ROC/AUC: [0.80523458 0.89871971 0.94978908 0.89019876 0.76726203]

=======교차검증 평균값=======
CV_Accuracy_mean: 0.781
CV_Precision_mean: 0.759
CV_Recall_mean: 0.825
CV_F1_스코어_mean: 0.791
CV_ROC_AUC_mean: 0.862


# 결과

In [44]:
default

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier
Accuracy,0.693,0.791,0.767,0.778,0.692,0.759,0.792
Precision,0.572,0.690,0.666,0.698,0.571,0.650,0.690
Recall,0.658,0.785,0.736,0.698,0.658,0.747,0.789
F1,0.612,0.734,0.699,0.698,0.611,0.695,0.736
ROC AUC,0.686,0.790,0.761,0.761,0.685,0.756,0.791


In [45]:
best

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier
Accuracy,0.693,0.792,0.772,0.782,0.692,0.76,0.791
Precision,0.572,0.673,0.669,0.687,0.571,0.653,0.687
Recall,0.658,0.845,0.75,0.751,0.658,0.744,0.795
ROC AUC,0.686,0.803,0.767,0.776,0.685,0.757,0.792
Best Parameters,"{'solver': 'liblinear', 'penalty': 'l2', 'C': ...","{'n_estimators': 50, 'min_samples_split': 4, '...","{'n_estimators': 240, 'learning_rate': 0.75}","{'n_estimators': 30, 'max_samples': 0.4, 'max_...","{'kernel': 'linear', 'gamma': 0.1, 'C': 0.1}","{'kernel': 'rbf', 'gamma': 0.05880000000000000...","{'n_estimators': 120, 'learning_rate': 0.1}"


In [46]:
validation

,LogisticRegression,RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,SVC_linear,SVC_rbf,LGBMClassifier,XGBClassifier
Accuracy,0.705,0.758,0.765,0.755,0.704,0.754,0.772,0.781
Precision,0.705,0.726,0.764,0.750,0.706,0.746,0.747,0.759
Recall,0.700,0.848,0.771,0.790,0.694,0.767,0.828,0.825
F1,0.701,0.780,0.767,0.756,0.698,0.756,0.785,0.791
ROC AUC,0.775,0.844,0.852,0.836,0.774,0.833,0.852,0.862
